# TensorFlow GPU avec Azure ML

<img src='https://github.com/retkowsky/images/blob/master/AzureMLservicebanniere.png?raw=true'>

In [42]:
import sys
sys.version

'3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) \n[GCC 7.3.0]'

In [43]:
import datetime
maintenant = datetime.datetime.now()
print("Date du run = ", maintenant)

Date du run =  2020-10-13 09:17:42.018808


In [44]:
import azureml.core
print("Version Azure ML =", azureml.core.VERSION)

Version Azure ML = 1.15.0


## 1. Workspace

In [45]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()

## 2. AML Compute GPU

Les tailles NC, NCv2 et NCv3 sont optimisées pour les algorithmes et les applications nécessitant beaucoup de ressources réseau et de calculs. 

En voici quelques exemples : les applications et les simulations CUDA et OpenCL, l’intelligence artificielle et l’apprentissage profond. Équipée du GPU Tesla V100 de NVIDIA, la série NCv3 est axée sur les charges de travail informatiques à hautes performances. 

- La série NC utilise le processeur Intel Xeon E5-2690 v3 2.60GHz v3 (Haswell) et les machines virtuelles de la série NCv2 et NCv3 sont dotées du processeur Intel Xeon E5-2690 v4 (Broadwell).

- ND et NDv2La série ND est destinée à l’exécution de scénarios d’apprentissage et d’inférence pour le Deep Learning. Elle utilise le GPU NVIDIA Tesla P40 et le processeur Intel Xeon E5-2690 v4 (Broadwell). La série NDv2 utilise le processeur Intel Xeon Platinum 8168 (Skylake).

- Les tailles NV et NVv3 sont optimisées et conçues pour la visualisation à distance, la diffusion en continu, les jeux, l’encodage et les scénarios de VDI utilisant des infrastructures comme OpenGL ou DirectX. Ces machines virtuelles reposent sur le GPU Tesla M60 de NVIDIA.

- Les tailles NVv4 sont optimisées et conçues pour l’infrastructure VDI et la visualisation à distance. Avec des GPU partitionnés, NVv4 offre la taille adaptée aux charges de travail nécessitant des ressources GPU plus petites. Ces machines virtuelles sont associées au GPU AMD Radeon Instinct MI25.

https://docs.microsoft.com/fr-fr/azure/virtual-machines/windows/sizes-gpu

In [46]:
from azureml.core.compute import ComputeTarget, AmlCompute

AmlCompute.supported_vmsizes(workspace = ws)

[{'name': 'Standard_D1_v2',
  'vCPUs': 1,
  'gpus': 0,
  'memoryGB': 3.5,
  'maxResourceVolumeMB': 51200},
 {'name': 'Standard_D2_v2',
  'vCPUs': 2,
  'gpus': 0,
  'memoryGB': 7.0,
  'maxResourceVolumeMB': 102400},
 {'name': 'Standard_D3_v2',
  'vCPUs': 4,
  'gpus': 0,
  'memoryGB': 14.0,
  'maxResourceVolumeMB': 204800},
 {'name': 'Standard_D4_v2',
  'vCPUs': 8,
  'gpus': 0,
  'memoryGB': 28.0,
  'maxResourceVolumeMB': 409600},
 {'name': 'Standard_D11_v2',
  'vCPUs': 2,
  'gpus': 0,
  'memoryGB': 14.0,
  'maxResourceVolumeMB': 102400},
 {'name': 'Standard_D12_v2',
  'vCPUs': 4,
  'gpus': 0,
  'memoryGB': 28.0,
  'maxResourceVolumeMB': 204800},
 {'name': 'Standard_D13_v2',
  'vCPUs': 8,
  'gpus': 0,
  'memoryGB': 56.0,
  'maxResourceVolumeMB': 409600},
 {'name': 'Standard_D14_v2',
  'vCPUs': 16,
  'gpus': 0,
  'memoryGB': 112.0,
  'maxResourceVolumeMB': 819200},
 {'name': 'Standard_DS1_v2',
  'vCPUs': 1,
  'gpus': 0,
  'memoryGB': 3.5,
  'maxResourceVolumeMB': 7168},
 {'name': 'Standar

In [50]:
%%time
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "monclustergpu"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           min_nodes=1,
                                                           max_nodes=8
                                                           )

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

Creating a new compute target...
Creating
SucceededCurrent provisioning state of AmlCompute is "Deleting"

.........Current provisioning state of AmlCompute is "Deleting"

........
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
CPU times: user 714 ms, sys: 58.8 ms, total: 773 ms
Wall time: 1min 49s


In [51]:
liste = ws.compute_targets
for liste in liste:
    print("Ressources Compute du workspace :", liste)

Ressources Compute du workspace : nbookinstance
Ressources Compute du workspace : monclustercpu
Ressources Compute du workspace : cpucluster
Ressources Compute du workspace : cpuclusterd2
Ressources Compute du workspace : clustergpu
Ressources Compute du workspace : monclustergpu


In [52]:
compute_target.get_status().serialize()

{'currentNodeCount': 1,
 'targetNodeCount': 1,
 'nodeStateCounts': {'preparingNodeCount': 0,
  'runningNodeCount': 0,
  'idleNodeCount': 1,
  'unusableNodeCount': 0,
  'leavingNodeCount': 0,
  'preemptedNodeCount': 0},
 'allocationState': 'Steady',
 'allocationStateTransitionTime': '2020-10-13T09:22:12.180000+00:00',
 'errors': None,
 'creationTime': '2020-10-13T09:20:32.042760+00:00',
 'modifiedTime': '2020-10-13T09:20:48.072220+00:00',
 'provisioningState': 'Succeeded',
 'provisioningStateTransitionTime': None,
 'scaleSettings': {'minNodeCount': 1,
  'maxNodeCount': 8,
  'nodeIdleTimeBeforeScaleDown': 'PT120S'},
 'vmPriority': 'Dedicated',
 'vmSize': 'STANDARD_NC6'}

In [53]:
# Statut du compute server
compute_target.list_nodes()

[{'nodeId': 'tvmps_2274911ec175e9dad95494c2f077cba07e648d2a266906edc23c2192b2beeed0_d',
  'port': 50000,
  'publicIpAddress': '52.149.78.43',
  'privateIpAddress': '10.0.0.4',
  'nodeState': 'idle'}]

## 3. Données

In [54]:
from azureml.core.dataset import Dataset
web_paths = ['http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
             'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
             'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
             'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz'
            ]
dataset = Dataset.File.from_files(path = web_paths)

In [55]:
dataset = dataset.register(workspace = ws,
                           name = 'mnist dataset',
                           description='training and test dataset',
                           create_new_version=True)

In [56]:
dataset.to_path()

['/http%3A/%2Fyann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
 '/http%3A/%2Fyann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
 '/http%3A/%2Fyann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
 '/http%3A/%2Fyann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz']

## 4. Création projet et expérimentation

In [60]:
import os
script_folder = './tf-resume-training'
os.makedirs(script_folder, exist_ok=True)

In [61]:
from azureml.core import Experiment

experiment_name = 'Workshop-TensorFlowGPU'
experiment = Experiment(ws, name=experiment_name)

## 5. Création et exécution estimator TensorFlow

### Visualisation du code python :

In [62]:
with open(os.path.join(script_folder, './tf_mnist_with_checkpoint.py'), 'r') as f:
    print(f.read())

# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.

import numpy as np
import utils
import argparse
import os
import re
import tensorflow as tf
import glob

from azureml.core import Run
from utils import load_data

print("TensorFlow version:", tf.__version__)

parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')

parser.add_argument('--resume-from', type=str, default=None,
                    help='location of the model or checkpoint files from where to resume the training')
args = parser.parse_args()


previous_model_location = args.resume_from
# You can also use environment variable to get the model/checkpoint files location
# previous_model_location = os.path.expandvars(os.getenv("AZUREML_DATAREFERENCE_MODEL_LOCATION", None))

data_folder = args.data_folder
print('Data folder:', data_folder)

# load train and test set into numpy arrays
# note we scale the pi

In [63]:
from azureml.train.dnn import TensorFlow

script_params={
    '--data-folder': dataset.as_named_input('mnist').as_mount()
}

estimator= TensorFlow(source_directory=script_folder,
                      compute_target=compute_target,
                      script_params=script_params,
                      entry_script='tf_mnist_with_checkpoint.py',
                      use_gpu=True,
                      pip_packages=['azureml-dataprep[pandas,fuse]'])

WARNING - framework_version is not specified, defaulting to version 1.13.


### Exécution du Run

In [64]:
tags = {"Type": "test" , "Langage" : "Python" , "Framework"  : "TensorFlow"}

In [65]:
run = experiment.submit(estimator, tags=tags)
print(run)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.
WARNING - If 'arguments' has been provided here and arguments have been specified in 'run_config', 'arguments' provided in ScriptRunConfig initialization will take precedence.


Run(Experiment: Workshop-TensorFlowGPU,
Id: Workshop-TensorFlowGPU_1602581349_4fd8cfc9,
Type: azureml.scriptrun,
Status: Queued)


### Widget pour suivi de l'exécution du run

In [66]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [75]:
print("Status =", run.get_status())

Status = Completed


In [76]:
run.get_details()

{'runId': 'Workshop-TensorFlowGPU_1602581349_4fd8cfc9',
 'target': 'monclustergpu',
 'status': 'Completed',
 'startTimeUtc': '2020-10-13T09:29:26.19161Z',
 'endTimeUtc': '2020-10-13T09:32:58.438591Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '31bdf40f-56f1-4107-a052-76bd089365d8',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '52443a4f-8030-4286-82bf-19641b575bca'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'mnist', 'mechanism': 'Mount'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'tf_mnist_with_checkpoint.py',
  'command': [],
  'useAbsolutePath': False,
  'arguments': ['--data-folder', 'DatasetConsumptionConfig:mnist'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'monclustergpu',
  'dataReferences': {},
  'data': {'mnist': {'dataLocation': {'dataset': {'id': 

### Capture d'écran des résultats du run :

<img src="https://github.com/retkowsky/images/blob/master/GPUTF.jpg?raw=true">

### Liste des métriques du run

In [77]:
run.get_metrics()

{'training_acc': [0.9800000190734863,
  0.9200000166893005,
  0.9200000166893005,
  0.9399999976158142,
  0.9200000166893005,
  0.9800000190734863,
  0.9399999976158142,
  0.9800000190734863,
  0.9800000190734863,
  0.9599999785423279,
  0.8999999761581421],
 'validation_acc': [0.8948000073432922,
  0.914900004863739,
  0.9261999726295471,
  0.9294000267982483,
  0.9359999895095825,
  0.9426000118255615,
  0.9447000026702881,
  0.9488000273704529,
  0.9523000121116638,
  0.9549000263214111,
  0.9559000134468079],
 'final_acc': 0.9559000134468079}

### Visualisation des métriques dans l'expérimentation depuis Azure ML Studio

In [78]:
experiment

Name,Workspace,Report Page,Docs Page
Workshop-TensorFlowGPU,workshoplcl,Link to Azure Machine Learning studio,Link to Documentation


### Suppression compute instance

In [79]:
# Statut du compute server
compute_target.list_nodes()

[{'nodeId': 'tvmps_2274911ec175e9dad95494c2f077cba07e648d2a266906edc23c2192b2beeed0_d',
  'port': 50000,
  'publicIpAddress': '52.149.78.43',
  'privateIpAddress': '10.0.0.4',
  'nodeState': 'idle'}]

In [80]:
# Suppression du cluster
compute_target.delete()

In [81]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, "(" , ct.type, ") :", ct.provisioning_state)

monclustergpu ( AmlCompute ) : Deleting
nbookinstance ( ComputeInstance ) : Succeeded
monclustercpu ( AmlCompute ) : Succeeded
cpucluster ( AmlCompute ) : Succeeded
cpuclusterd2 ( AmlCompute ) : Succeeded
clustergpu ( AmlCompute ) : Succeeded


<img src="https://github.com/retkowsky/images/blob/master/Powered-by-MS-Azure-logo-v2.png?raw=true" height="300" width="300">